In [1]:
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score
import multiprocessing
from tqdm.auto import tqdm
import os
import numpy as np
import scipy.sparse as sp
import re

/home/anonymous/miniconda3/envs/belka/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def extract_uuid(filename):
    # UUID pattern: 8-4-4-4-12 hexadecimal characters
    uuid_pattern = r'[0-9a-fA-F]{8}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{12}'
    
    # Search for the UUID pattern in the filename
    match = re.search(uuid_pattern, filename)
    if match:
        return match.group()  # Return the found UUID
    else:
        return None  # No UUID found
    
def concat_and_save(prefix, directory, resulting_file):
    files = [f for f in os.listdir(directory) if f.endswith('.npz') and f.startswith(prefix)]
    uuids = [extract_uuid(x) for x in files]

    # Initialize a list to hold the loaded sparse matrices
    matrices = []
    
    # Load each matrix and append to the list
    for file in tqdm(uuids):
        file_path = os.path.join(directory, prefix+file+'.npz')
        matrix = sp.load_npz(file_path)
        matrices.append(matrix)
    
    # Concatenate all COO matrices over rows
    concatenated_matrix = sp.vstack(matrices, format='coo')
    sp.save_npz(f'/mnt/fastssd/belka_data/ready_data/{resulting_file}_m', concatenated_matrix)

    files = [f for f in os.listdir(directory) if f.endswith('.npy') and f.startswith(prefix)]
    # Initialize a list to hold the loaded sparse matrices
    matrices = []
    
    # Load each matrix and append to the list
    for file in tqdm(uuids):
        file_path = os.path.join(directory, prefix+file+'.npl.npy')
        matrix = np.load(file_path)
        matrices.append(matrix)
    
    # Concatenate all COO matrices over rows
    concatenated_matrix = np.hstack(matrices)
    np.save(f'/mnt/fastssd/belka_data/ready_data/{resulting_file}_t', concatenated_matrix)


def concatenate_coo_matrices(directory):
    # List all .npz files in the specified directory
    concat_and_save('unique_test_', directory,'unique_test_')
    concat_and_save('random_test_', directory,'random_test_')
    concat_and_save('train_', directory,'train_')
    

# Example usage
directory_path = '/mnt/fastssd/belka_data/train_split_test/'
concatenated_matrix = concatenate_coo_matrices(directory_path)
if concatenated_matrix is not None:
    print("Successfully concatenated COO matrices.")
    print(concatenated_matrix)
else:
    print("No matrices found or concatenated.")

100%|██████████| 944/944 [00:02<00:00, 367.66it/s]


No matrices found or concatenated.
